**CHECKING THE WEATHER OF THE DAY, COULD BE USED BY STAFF**


In [2]:
import requests
from datetime import datetime, timezone, timedelta
import pytz
import sys

LATITUDE = 41.9742        #for chicago/ ORD airport
LONGITUDE = -87.9073
OPEN_METEO = "https://api.open-meteo.com/v1/forecast"
REQUEST_TIMEOUT = 12
CHI_TZ = pytz.timezone("America/Chicago")

# Weather code map (short)
WEATHER_CODE_MAP = {
    0: "Clear sky", 1: "Mainly clear", 2: "Partly cloudy", 3: "Overcast",
    45: "Fog", 48: "Depositing rime fog",
    51: "Light drizzle", 53: "Moderate drizzle", 55: "Dense drizzle",
    61: "Slight rain", 63: "Moderate rain", 65: "Heavy rain",
    71: "Slight snow", 73: "Moderate snow", 75: "Heavy snow",
    80: "Slight rain showers", 81: "Moderate rain showers", 82: "Violent rain showers",
    95: "Thunderstorm", 96: "Thunderstorm with slight hail", 99: "Thunderstorm with heavy hail",
}


def build_url(lat, lon, tz="America/Chicago"):
    # request hourly temperature, weathercode, precipitation, precip prob, cloudcover, windspeed
    return (f"{OPEN_METEO}?latitude={lat}&longitude={lon}"
            f"&current_weather=true"
            f"&hourly=temperature_2m,weathercode,precipitation,precipitation_probability,cloudcover,windspeed_10m"
            f"&timezone={tz}")

def fetch_weather(lat=LATITUDE, lon=LONGITUDE, tz="America/Chicago"):
    url = build_url(lat, lon, tz)
    resp = requests.get(url, timeout=REQUEST_TIMEOUT)
    resp.raise_for_status()
    return resp.json()

# parse iso -> naive datetime (keeps local tz if API returned naive local strings)
def parse_iso_to_naive(iso_str):
    if iso_str is None:
        return None
    try:
        dt = datetime.fromisoformat(iso_str)
        # If it has tzinfo, convert to CHI tz and return naive local time
        if dt.tzinfo is not None:
            return dt.astimezone(CHI_TZ).replace(tzinfo=None)
        # otherwise treat as local America/Chicago naive
        return dt
    except Exception:
        for fmt in ("%Y-%m-%dT%H:%M:%S", "%Y-%m-%dT%H:%M"):
            try:
                return datetime.strptime(iso_str, fmt)
            except Exception:
                continue
    return None

def classify(desc):
    if desc is None:
        return "unknown"
    d = desc.lower()
    if "clear" in d and "mainly" not in d:
        return "sunny"
    if "mainly clear" in d:
        return "mostly clear"
    if "partly" in d:
        return "partly cloudy"
    if "overcast" in d or "cloud" in d:
        return "cloudy"
    if "drizzle" in d or "rain" in d or "shower" in d:
        return "rainy"
    if "snow" in d:
        return "snowy"
    if "thunder" in d:
        return "thunderstorm"
    if "fog" in d or "mist" in d:
        return "foggy"
    return desc

def intensity_from_precip(mm):
    try:
        mmv = float(mm)
    except Exception:
        return "unknown"
    if mmv <= 0:
        return "no precipitation"
    if mmv < 2:
        return "light precipitation"
    if mmv < 10:
        return "moderate precipitation"
    return "heavy precipitation"

def is_daytime_for_local_naive(dt_naive):
    """
    dt_naive: naive datetime representing America/Chicago local time (no tzinfo).
    Returns True if local hour is between 6 and 17 inclusive (6:00-17:59).
    """
    if dt_naive is None:
        return True
    try:
        # localize naive into CHI tz
        local_dt = CHI_TZ.localize(dt_naive)
        h = local_dt.hour
        return 6 <= h < 18
    except Exception:
        # fallback to now
        now = datetime.now(CHI_TZ)
        return 6 <= now.hour < 18


def two_hour_forecast_for_today(api_json):
    """
    Return list of forecast entries for today at 2-hour intervals (00,02,...,22) local Chicago time.
    Each entry: {time, temp, code, desc, condition, precip_mm, precip_prob, cloud_pct, wind_kmh, intensity}
    """
    hourly = api_json.get("hourly", {}) or {}
    times = hourly.get("time", [])
    temps = hourly.get("temperature_2m", [])
    codes = hourly.get("weathercode", [])
    precs = hourly.get("precipitation", [])
    prec_probs = hourly.get("precipitation_probability", [])
    clouds = hourly.get("cloudcover", [])
    winds = hourly.get("windspeed_10m", [])

    # build list of (idx, local_naive_dt) for all hourly entries
    idx_dt_pairs = []
    for i, tstr in enumerate(times):
        dt_naive = parse_iso_to_naive(tstr)
        if dt_naive is None:
            continue
        # dt_naive is local Chicago naive (per parse_iso_to_naive)
        idx_dt_pairs.append((i, dt_naive))

    # get today's date in Chicago (local)
    today_local = datetime.now(CHI_TZ).date()

    # collect indices for today's date with hour in 0..23
    todays_indices = [(i, dt) for i, dt in idx_dt_pairs if dt.date() == today_local]
    if not todays_indices:
        # fallback: take next 24 hours from first hourly element
        if idx_dt_pairs:
            base_idx = idx_dt_pairs[0][0]
            todays_indices = [(i, parse_iso_to_naive(times[i]) or datetime(1900,1,1)) for i in range(base_idx, base_idx+24) if i < len(times)]
        else:
            return []

    # we want hours 0,2,4,...,22 local — but ensure those exist: filter by dt.hour % 2 == 0
    # This will produce entries at 00:00, 02:00, ... as available for today.
    two_hour_entries = []
    for i, dt in todays_indices:
        if dt.hour % 2 == 0:
            # gather values safely
            def safe_get(arr, idx):
                try:
                    return arr[idx]
                except Exception:
                    return None
            temp = safe_get(temps, i)
            code = safe_get(codes, i)
            prec = safe_get(precs, i)
            pprob = safe_get(prec_probs, i)
            cloud = safe_get(clouds, i)
            wind = safe_get(winds, i)
            try:
                code_int = int(code) if code is not None else None
            except Exception:
                code_int = None
            desc = WEATHER_CODE_MAP.get(code_int, "Unknown")
            condition = classify(desc)
            # adjust for night/day using this dt
            if condition == "sunny" and not is_daytime_for_local_naive(dt):
                condition = "clear night"
            if condition == "mostly clear" and not is_daytime_for_local_naive(dt):
                condition = "mostly clear (night)"
            intensity = intensity_from_precip(prec)
            two_hour_entries.append({
                "time": dt.strftime("%Y-%m-%dT%H:%M"),
                "hour": dt.hour,
                "temperature_c": temp,
                "weather_code": code_int,
                "description": desc,
                "condition": condition,
                "precipitation_mm": prec,
                "precipitation_probability_pct": pprob,
                "cloudcover_pct": cloud,
                "wind_kmh": wind,
                "intensity": intensity,
            })

    # Ensure sorted by hour
    two_hour_entries.sort(key=lambda x: x["hour"])
    return two_hour_entries


def print_two_hour_forecast(entries):
    if not entries:
        print("No 2-hour forecast entries found for today.")
        return
    print("\n2-hour interval forecast (Chicago local):")
    print("-" * 72)
    print(f"{'Time':16} {'Cond.':15} {'Temp (°C)':10} {'Precip(mm)':12} {'Cloud%':7} {'Wind(km/h)':10}")
    print("-" * 72)
    for e in entries:
        time = e["time"]
        cond = e["condition"]
        temp = f"{e['temperature_c']:.1f}" if isinstance(e["temperature_c"], (int, float)) else (str(e["temperature_c"]) if e["temperature_c"] is not None else "N/A")
        prec = f"{float(e['precipitation_mm']):.2f}" if e["precipitation_mm"] not in (None, "") else "N/A"
        cloud = f"{e['cloudcover_pct']}" if e["cloudcover_pct"] is not None else "N/A"
        wind = f"{e['wind_kmh']}" if e["wind_kmh"] is not None else "N/A"
        print(f"{time:16} {cond:15} {temp:10} {prec:12} {cloud:7} {wind:10}")
    print("-" * 72 + "\n")


def print_summary_short(summary):
    # basic current summary using current_weather if available
    cur_time = summary.get("api_time")
    try:
        local_now = datetime.now(CHI_TZ)
        local_str = local_now.strftime("%A, %B %d, %Y %I:%M %p %Z")
    except Exception:
        local_str = datetime.now(timezone.utc).strftime("%Y-%m-%d %H:%M UTC")
    print()
    print(f"Local Date & Time (Chicago): {local_str}")
    print(f"API time: {summary.get('api_time')}")
    print(f"Temperature: {summary.get('temperature_c')} °C")
    print(f"Condition: {summary.get('condition')} ({summary.get('description')})")
    print(f"Precipitation (mm): {summary.get('precipitation_mm')} ({summary.get('intensity_label')})")
    print()

# ----------------
def run_all():
    try:
        data = fetch_weather()
    except Exception as e:
        print("Error fetching data:", e)
        return 1

    # Using earlier interpret (compact) for current-like summary
    # reuse parse_iso_to_naive and nearest matching pattern similar to interpret()
    try:
        # Build a simple current summary (closest hour)
        current = data.get("current_weather", {}) or {}
        cur_time_s = current.get("time")
        cur_dt = parse_iso_to_naive(cur_time_s)
        hourly_times = data.get("hourly", {}).get("time", [])
        hourly_dt_list = [parse_iso_to_naive(t) or datetime(1900,1,1) for t in hourly_times]
        # nearest
        best_idx = 0
        if hourly_dt_list and cur_dt:
            best_idx = min(range(len(hourly_dt_list)), key=lambda i: abs((hourly_dt_list[i] - cur_dt).total_seconds()))
        def safe(arr, idx):
            try:
                return arr[idx]
            except Exception:
                return None
        summary = {
            "api_time": cur_time_s,
            "temperature_c": safe(data.get("hourly", {}).get("temperature_2m", []), best_idx) or current.get("temperature"),
            "description": WEATHER_CODE_MAP.get(int(safe(data.get("hourly", {}).get("weathercode", []), best_idx) or current.get("weathercode") or -1), "Unknown"),
            "condition": None,
            "precipitation_mm": safe(data.get("hourly", {}).get("precipitation", []), best_idx),
            "intensity_label": None
        }
        desc = summary["description"]
        cond = classify(desc)
        if cond == "sunny" and not is_daytime_for_local_naive(hourly_dt_list[best_idx] if best_idx < len(hourly_dt_list) else cur_dt):
            cond = "clear night"
        summary["condition"] = cond
        summary["intensity_label"] = intensity_from_precip(summary["precipitation_mm"])
        print_summary_short(summary)
    except Exception:
        pass

    # two-hour forecast
    two_hour_entries = two_hour_forecast_for_today(data)
    print_two_hour_forecast(two_hour_entries)
    return 0

if __name__ == "__main__":
    exit_code = run_all()
    sys.exit(exit_code)



Local Date & Time (Chicago): Sunday, October 05, 2025 08:29 AM CDT
API time: 2025-10-05T08:15
Temperature: 19.1 °C
Condition: Unknown (Unknown)
Precipitation (mm): 0.0 (no precipitation)


2-hour interval forecast (Chicago local):
------------------------------------------------------------------------
Time             Cond.           Temp (°C)  Precip(mm)   Cloud%  Wind(km/h)
------------------------------------------------------------------------
2025-10-05T00:00 clear night     23.9       0.00         0       12.6      
2025-10-05T02:00 clear night     22.8       0.00         0       12.3      
2025-10-05T04:00 clear night     20.0       0.00         0       9.1       
2025-10-05T06:00 sunny           18.6       0.00         0       5.8       
2025-10-05T08:00 sunny           19.1       0.00         1       11.5      
2025-10-05T10:00 sunny           24.2       0.00         1       17.3      
2025-10-05T12:00 cloudy          26.9       0.00         100     23.8      
2025-10-05T14:

SystemExit: 0